In [1]:
import os
import tensorflow  as tf
import time

In [2]:
def decode(serialized_example):
    alphabets=list()
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'cho_label':tf.FixedLenFeature([], tf.int64),
          'jung_label':tf.FixedLenFeature([], tf.int64),
          'jong_label':tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    cho_label=tf.cast(features['cho_label'],tf.int64)
    jung_label=tf.cast(features['jung_label'],tf.int64)
    jong_label=tf.cast(features['jong_label'],tf.int64)
    
    return image,label, cho_label, jung_label, jong_label

In [3]:
batch_size=100
test_file_name = "Shuffled_with_alphabet_PE92_Test.tfrecord"
train_file_name = "Shuffled_with_alphabet_PE92_Train.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

In [4]:
train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [5]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_label_cho = tf.placeholder(tf.int64,[None])
input_label_jung = tf.placeholder(tf.int64,[None])
input_label_jong = tf.placeholder(tf.int64,[None])
input_labels= tf.placeholder(tf.int64,[None])


is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)
print(input_images)


Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)


In [6]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=is_training)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [7]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=is_training)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [8]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=is_training)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [9]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=is_training)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W5 = tf.Variable(tf.random_normal([2, 2, 256, 512], stddev=0.01))
L5 = tf.nn.conv2d(pool4,W5, strides=[1,1,1,1], padding = 'SAME')
bn5 = tf.layers.batch_normalization(L5, training=is_training)
at5 = tf.nn.relu(bn5)
pool5 =  tf.nn.max_pool(at5, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

pool6 =  tf.nn.max_pool(pool5, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)
print(pool6)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)
Tensor("MaxPool_5:0", shape=(?, 1, 1, 512), dtype=float32)


In [10]:
fc = tf.reshape(pool6,[-1,512])

W_cho1 = tf.Variable(tf.random_normal([512, 48], stddev=0.01))
W_jung1 = tf.Variable(tf.random_normal([512, 48], stddev=0.01))
W_jong1 = tf.Variable(tf.random_normal([512, 48], stddev=0.01))

fc_cho = tf.matmul(fc, W_cho1)
fc_cho = tf.layers.batch_normalization(fc_cho, training=is_training)
fc_cho = tf.nn.relu(fc_cho)
fc_cho= tf.nn.dropout(fc_cho,keep_prob)

fc_jung = tf.matmul(fc, W_jung1)
fc_jung = tf.layers.batch_normalization(fc_jung, training=is_training)
fc_jung = tf.nn.relu(fc_jung)
fc_jung= tf.nn.dropout(fc_jung,keep_prob)

fc_jong = tf.matmul(fc, W_jong1)
fc_jong = tf.layers.batch_normalization(fc_jong, training=is_training)
fc_jong = tf.nn.relu(fc_jong)
fc_jong= tf.nn.dropout(fc_jong,keep_prob)
print(fc)

Tensor("Reshape:0", shape=(?, 512), dtype=float32)


In [11]:
W_cho2 = tf.Variable(tf.random_normal([48,19],stddev=0.01))
W_jung2 = tf.Variable(tf.random_normal([48,21],stddev=0.01))
W_jong2 = tf.Variable(tf.random_normal([48,28],stddev=0.01))

logit_cho = tf.matmul(fc_cho, W_cho2)
logit_jung = tf.matmul(fc_jung, W_jung2)
logit_jong = tf.matmul(fc_jong, W_jong2)

print(logit_cho)
print(logit_jung)
print(logit_jong)

Tensor("MatMul_3:0", shape=(?, 19), dtype=float32)
Tensor("MatMul_4:0", shape=(?, 21), dtype=float32)
Tensor("MatMul_5:0", shape=(?, 28), dtype=float32)


In [12]:
W_word = tf.Variable(tf.random_normal([512, 1024], stddev=0.01))
fc_word = tf.matmul(fc, W_word)
fc_word = tf.layers.batch_normalization(fc_word, training=is_training)
fc_word = tf.nn.relu(fc_word)
fc_word= tf.nn.dropout(fc_word, keep_prob)

W_word2 = tf.Variable(tf.random_normal([512, 2350], stddev=0.01))
logit_word = tf.matmul(fc, W_word2)

cross_entropy_word = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_word, labels=input_labels)
loss_word = tf.reduce_mean(cross_entropy_word)

In [13]:
cross_entropy_cho = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_cho, labels=input_label_cho)
cross_entropy_jung = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_jung, labels=input_label_jung)
cross_entropy_jong = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_jong, labels=input_label_jong)

loss_cho = tf.reduce_mean(cross_entropy_cho)
cho_loss = tf.summary.scalar("Loss_cho", loss_cho)
loss_jung = tf.reduce_mean(cross_entropy_jung)
jung_loss = tf.summary.scalar("Loss_jung",loss_jung)
loss_jong = tf.reduce_mean(cross_entropy_jong)
jong_loss = tf.summary.scalar("Loss_jong",loss_jong)

total_loss = 3*((loss_cho*0.4) + (loss_jung*0.4) + (loss_jong*0.2))+loss_word

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(total_loss)
loss_summ=tf.summary.scalar("Loss",total_loss)
print(total_loss)

Tensor("add_2:0", shape=(), dtype=float32)


In [14]:
argmax_cho = tf.argmax(logit_cho,1)
argmax_jung = tf.argmax(logit_jung,1)
argmax_jong = tf.argmax(logit_jong,1)
print(argmax_cho)
prediction = tf.stack([argmax_cho,argmax_jung,argmax_jong],1)

Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [15]:
total_labels = tf.stack([input_label_cho, input_label_jung, input_label_jong],1)
equal_check = tf.equal(prediction,total_labels)
cast_equal = tf.cast(equal_check, tf.float32)
reduce_min = tf.reduce_min(cast_equal,1)

accuracy = tf.reduce_mean(reduce_min)
test_accuracy = tf.reduce_sum(reduce_min)
acc_summ=tf.summary.scalar("Accuracy",accuracy)

In [16]:
correct_prediction2 = tf.equal(tf.argmax(logit_word,1), input_labels)
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
test_accuracy2 = tf.reduce_sum(tf.cast(correct_prediction2, tf.float32))
acc_summ2=tf.summary.scalar("Accuracy",accuracy2)

In [17]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    
    merged_summary = tf.summary.merge([loss_summ, acc_summ])
    train_writer = tf.summary.FileWriter("./logs/CNNV4/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV4/PE92/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    
    #Train Model
    
    start_time = time.time()
    global_step=0
    
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_label, train_cho, train_jung, train_jong])
            
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels: batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy2, total_loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary, feed_dict={input_images: batch_images, input_labels:batch_labels, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/PE92/CNNV4/CNNV4_PE92.ckpt")
        
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_label, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss=sess.run([test_accuracy2, total_loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong,keep_prob:1.0, is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_labels)
            
        loss_sum=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))


    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 16.944181
Epoch :0 | step 100, training accuracy: 0.010000, train loss: 16.903893
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 16.650202
Epoch :0 | step 300, training accuracy: 0.000000, train loss: 15.895990
Epoch :0 | step 400, training accuracy: 0.000000, train loss: 14.483009
Epoch :0 | step 500, training accuracy: 0.010000, train loss: 13.280025
Epoch :0 | step 600, training accuracy: 0.050000, train loss: 11.971066
Epoch :0 | step 700, training accuracy: 0.060000, train loss: 11.134963
Epoch :0 | step 800, training accuracy: 0.130000, train loss: 10.375088
Epoch :0 | step 900, training accuracy: 0.230000, train loss: 9.030272
Epoch :0 | step 1000, training accuracy: 0.140000, train loss: 9.156956
Epoch :0 | step 1100, training accuracy: 0.250000, train loss: 7.868978
Epoch :0 | step 1200, training accuracy: 0.340000, train loss: 7.726354
Epoch :0 | step 1300, training accuracy: 0.

Epoch :5 | step 500, training accuracy: 1.000000, train loss: 0.207741
Epoch :5 | step 600, training accuracy: 0.990000, train loss: 0.254429
Epoch :5 | step 700, training accuracy: 0.970000, train loss: 0.353768
Epoch :5 | step 800, training accuracy: 0.970000, train loss: 0.330194
Epoch :5 | step 900, training accuracy: 0.970000, train loss: 0.424999
Epoch :5 | step 1000, training accuracy: 0.970000, train loss: 0.347747
Epoch :5 | step 1100, training accuracy: 0.980000, train loss: 0.222939
Epoch :5 | step 1200, training accuracy: 0.970000, train loss: 0.328649
Epoch :5 | step 1300, training accuracy: 0.940000, train loss: 0.632560
Epoch :5 | step 1400, training accuracy: 0.950000, train loss: 0.460433
Epoch :5 | step 1500, training accuracy: 0.930000, train loss: 0.493537
Epoch :5 | step 1600, training accuracy: 0.990000, train loss: 0.242931
Epoch :5 | step 1700, training accuracy: 0.960000, train loss: 0.410558
Epoch :5 | step 1800, training accuracy: 0.990000, train loss: 0.4074

Epoch :10 | step 1000, training accuracy: 1.000000, train loss: 0.080085
Epoch :10 | step 1100, training accuracy: 0.970000, train loss: 0.158038
Epoch :10 | step 1200, training accuracy: 0.990000, train loss: 0.084857
Epoch :10 | step 1300, training accuracy: 0.990000, train loss: 0.234587
Epoch :10 | step 1400, training accuracy: 0.980000, train loss: 0.166533
Epoch :10 | step 1500, training accuracy: 0.990000, train loss: 0.235518
Epoch :10 | step 1600, training accuracy: 1.000000, train loss: 0.077447
Epoch :10 | step 1700, training accuracy: 0.980000, train loss: 0.151184
Epoch :10 | step 1800, training accuracy: 0.990000, train loss: 0.215732
Epoch :10 | step 1900, training accuracy: 1.000000, train loss: 0.064673
Epoch :10 | step 2000, training accuracy: 0.990000, train loss: 0.160351
Time : 0 : 37: :7.902429
Test Acc : 0.956797, Test Loss :0.060559
Epoch :11 | step 0, training accuracy: 0.990000, train loss: 0.111054
Epoch :11 | step 100, training accuracy: 0.990000, train loss

Epoch :15 | step 1400, training accuracy: 1.000000, train loss: 0.035362
Epoch :15 | step 1500, training accuracy: 0.990000, train loss: 0.089569
Epoch :15 | step 1600, training accuracy: 0.990000, train loss: 0.074336
Epoch :15 | step 1700, training accuracy: 0.980000, train loss: 0.067029
Epoch :15 | step 1800, training accuracy: 1.000000, train loss: 0.048819
Epoch :15 | step 1900, training accuracy: 0.990000, train loss: 0.187741
Epoch :15 | step 2000, training accuracy: 1.000000, train loss: 0.088177
Time : 0 : 54: :2.218444
Test Acc : 0.966027, Test Loss :0.015382
Epoch :16 | step 0, training accuracy: 0.990000, train loss: 0.068788
Epoch :16 | step 100, training accuracy: 0.990000, train loss: 0.042864
Epoch :16 | step 200, training accuracy: 1.000000, train loss: 0.054351
Epoch :16 | step 300, training accuracy: 0.990000, train loss: 0.031526
Epoch :16 | step 400, training accuracy: 1.000000, train loss: 0.140869
Epoch :16 | step 500, training accuracy: 0.980000, train loss: 0.

Epoch :20 | step 1800, training accuracy: 1.000000, train loss: 0.025727
Epoch :20 | step 1900, training accuracy: 1.000000, train loss: 0.053183
Epoch :20 | step 2000, training accuracy: 1.000000, train loss: 0.017356
Time : 1 : 10: :54.782360
Test Acc : 0.967993, Test Loss :0.407495
Epoch :21 | step 0, training accuracy: 1.000000, train loss: 0.076959
Epoch :21 | step 100, training accuracy: 1.000000, train loss: 0.030099
Epoch :21 | step 200, training accuracy: 1.000000, train loss: 0.021599
Epoch :21 | step 300, training accuracy: 1.000000, train loss: 0.026069
Epoch :21 | step 400, training accuracy: 0.990000, train loss: 0.052783
Epoch :21 | step 500, training accuracy: 1.000000, train loss: 0.056817
Epoch :21 | step 600, training accuracy: 1.000000, train loss: 0.026843
Epoch :21 | step 700, training accuracy: 0.990000, train loss: 0.068392
Epoch :21 | step 800, training accuracy: 1.000000, train loss: 0.042046
Epoch :21 | step 900, training accuracy: 1.000000, train loss: 0.067

Epoch :26 | step 100, training accuracy: 1.000000, train loss: 0.024769
Epoch :26 | step 200, training accuracy: 1.000000, train loss: 0.022240
Epoch :26 | step 300, training accuracy: 1.000000, train loss: 0.022305
Epoch :26 | step 400, training accuracy: 1.000000, train loss: 0.031375
Epoch :26 | step 500, training accuracy: 1.000000, train loss: 0.018370
Epoch :26 | step 600, training accuracy: 1.000000, train loss: 0.006954
Epoch :26 | step 700, training accuracy: 1.000000, train loss: 0.034436
Epoch :26 | step 800, training accuracy: 0.980000, train loss: 0.061287
Epoch :26 | step 900, training accuracy: 1.000000, train loss: 0.021701
Epoch :26 | step 1000, training accuracy: 1.000000, train loss: 0.030875
Epoch :26 | step 1100, training accuracy: 0.990000, train loss: 0.059068
Epoch :26 | step 1200, training accuracy: 1.000000, train loss: 0.028529
Epoch :26 | step 1300, training accuracy: 1.000000, train loss: 0.027559
Epoch :26 | step 1400, training accuracy: 1.000000, train lo

Epoch :31 | step 500, training accuracy: 1.000000, train loss: 0.014096
Epoch :31 | step 600, training accuracy: 1.000000, train loss: 0.016701
Epoch :31 | step 700, training accuracy: 1.000000, train loss: 0.079211
Epoch :31 | step 800, training accuracy: 1.000000, train loss: 0.006839
Epoch :31 | step 900, training accuracy: 1.000000, train loss: 0.005626
Epoch :31 | step 1000, training accuracy: 1.000000, train loss: 0.008315
Epoch :31 | step 1100, training accuracy: 1.000000, train loss: 0.012182
Epoch :31 | step 1200, training accuracy: 1.000000, train loss: 0.012792
Epoch :31 | step 1300, training accuracy: 0.990000, train loss: 0.048411
Epoch :31 | step 1400, training accuracy: 0.990000, train loss: 0.054456
Epoch :31 | step 1500, training accuracy: 1.000000, train loss: 0.010189
Epoch :31 | step 1600, training accuracy: 1.000000, train loss: 0.011476
Epoch :31 | step 1700, training accuracy: 1.000000, train loss: 0.024330
Epoch :31 | step 1800, training accuracy: 1.000000, trai

Epoch :36 | step 900, training accuracy: 0.990000, train loss: 0.044055
Epoch :36 | step 1000, training accuracy: 1.000000, train loss: 0.022474
Epoch :36 | step 1100, training accuracy: 1.000000, train loss: 0.010235
Epoch :36 | step 1200, training accuracy: 1.000000, train loss: 0.013823
Epoch :36 | step 1300, training accuracy: 1.000000, train loss: 0.031192
Epoch :36 | step 1400, training accuracy: 0.990000, train loss: 0.020698
Epoch :36 | step 1500, training accuracy: 1.000000, train loss: 0.004821
Epoch :36 | step 1600, training accuracy: 0.990000, train loss: 0.028323
Epoch :36 | step 1700, training accuracy: 1.000000, train loss: 0.011054
Epoch :36 | step 1800, training accuracy: 1.000000, train loss: 0.009385
Epoch :36 | step 1900, training accuracy: 1.000000, train loss: 0.030216
Epoch :36 | step 2000, training accuracy: 1.000000, train loss: 0.020337
Time : 2 : 4: :53.850624
Test Acc : 0.970813, Test Loss :0.012106
Epoch :37 | step 0, training accuracy: 1.000000, train loss

Epoch :41 | step 1300, training accuracy: 1.000000, train loss: 0.012409
Epoch :41 | step 1400, training accuracy: 1.000000, train loss: 0.010596
Epoch :41 | step 1500, training accuracy: 1.000000, train loss: 0.014031
Epoch :41 | step 1600, training accuracy: 0.990000, train loss: 0.117442
Epoch :41 | step 1700, training accuracy: 0.990000, train loss: 0.010403
Epoch :41 | step 1800, training accuracy: 1.000000, train loss: 0.013245
Epoch :41 | step 1900, training accuracy: 1.000000, train loss: 0.005302
Epoch :41 | step 2000, training accuracy: 1.000000, train loss: 0.006053
Time : 2 : 21: :46.274531
Test Acc : 0.967907, Test Loss :0.007825
Epoch :42 | step 0, training accuracy: 1.000000, train loss: 0.004922
Epoch :42 | step 100, training accuracy: 1.000000, train loss: 0.044310
Epoch :42 | step 200, training accuracy: 1.000000, train loss: 0.012901
Epoch :42 | step 300, training accuracy: 1.000000, train loss: 0.032357
Epoch :42 | step 400, training accuracy: 1.000000, train loss: 

Epoch :46 | step 1700, training accuracy: 1.000000, train loss: 0.033715
Epoch :46 | step 1800, training accuracy: 1.000000, train loss: 0.006371
Epoch :46 | step 1900, training accuracy: 1.000000, train loss: 0.004000
Epoch :46 | step 2000, training accuracy: 1.000000, train loss: 0.006068
Time : 2 : 38: :38.950453
Test Acc : 0.966283, Test Loss :0.258284
Epoch :47 | step 0, training accuracy: 1.000000, train loss: 0.012106
Epoch :47 | step 100, training accuracy: 1.000000, train loss: 0.004479
Epoch :47 | step 200, training accuracy: 1.000000, train loss: 0.006317
Epoch :47 | step 300, training accuracy: 0.990000, train loss: 0.015133
Epoch :47 | step 400, training accuracy: 1.000000, train loss: 0.005612
Epoch :47 | step 500, training accuracy: 1.000000, train loss: 0.044275
Epoch :47 | step 600, training accuracy: 1.000000, train loss: 0.009077
Epoch :47 | step 700, training accuracy: 1.000000, train loss: 0.004311
Epoch :47 | step 800, training accuracy: 1.000000, train loss: 0.00

In [18]:
train_file_name = "Shuffled_with_alphabet_HanDB_Train.tfrecord"
test_file_name = "Shuffled_with_alphabet_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [19]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary2 = tf.summary.merge([loss_summ, acc_summ])
    train_writer = tf.summary.FileWriter("./logs/CNNV4/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV4/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)

    start_time = time.time()
    global_step=0
    
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_label, train_cho, train_jung, train_jong])
            
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels: batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy, total_loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary2, feed_dict={input_images: batch_images, input_labels:batch_labels, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/HanDB/CNNV4/CNNV4_HanDB.ckpt")
        
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_label, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss=sess.run([test_accuracy, total_loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong,keep_prob:1.0, is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_labels)
            
        loss_sum=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
        
    print("Training is done.")


    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :6658
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 16.942886
Epoch :0 | step 100, training accuracy: 0.010000, train loss: 16.746086
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 16.653767
Epoch :0 | step 300, training accuracy: 0.020000, train loss: 16.088984
Epoch :0 | step 400, training accuracy: 0.110000, train loss: 13.434359
Epoch :0 | step 500, training accuracy: 0.170000, train loss: 12.878973
Epoch :0 | step 600, training accuracy: 0.180000, train loss: 11.119435
Epoch :0 | step 700, training accuracy: 0.250000, train loss: 10.869944
Epoch :0 | step 800, training accuracy: 0.220000, train loss: 8.617763
Epoch :0 | step 900, training accuracy: 0.230000, train loss: 7.627278
Epoch :0 | step 1000, training accuracy: 0.320000, train loss: 7.247664
Epoch :0 | step 1100, training accuracy: 0.380000, train loss: 6.281006
Epoch :0 | step 1200, training accuracy: 0.350000, train loss: 6.826477
Epoch :0 | step 1300, training accuracy: 0.4

Epoch :1 | step 4600, training accuracy: 0.940000, train loss: 0.392838
Epoch :1 | step 4700, training accuracy: 0.900000, train loss: 0.456713
Epoch :1 | step 4800, training accuracy: 0.920000, train loss: 0.555538
Epoch :1 | step 4900, training accuracy: 0.930000, train loss: 0.300700
Epoch :1 | step 5000, training accuracy: 0.840000, train loss: 1.056584
Epoch :1 | step 5100, training accuracy: 0.950000, train loss: 0.360591
Epoch :1 | step 5200, training accuracy: 0.900000, train loss: 0.682762
Epoch :1 | step 5300, training accuracy: 0.930000, train loss: 0.342169
Epoch :1 | step 5400, training accuracy: 0.920000, train loss: 0.460235
Epoch :1 | step 5500, training accuracy: 0.890000, train loss: 0.567014
Epoch :1 | step 5600, training accuracy: 0.920000, train loss: 0.476786
Epoch :1 | step 5700, training accuracy: 0.890000, train loss: 0.578912
Epoch :1 | step 5800, training accuracy: 0.930000, train loss: 0.460177
Epoch :1 | step 5900, training accuracy: 0.940000, train loss: 0

Epoch :3 | step 2500, training accuracy: 0.940000, train loss: 0.520031
Epoch :3 | step 2600, training accuracy: 0.950000, train loss: 0.458590
Epoch :3 | step 2700, training accuracy: 0.970000, train loss: 0.256228
Epoch :3 | step 2800, training accuracy: 0.930000, train loss: 0.552047
Epoch :3 | step 2900, training accuracy: 0.950000, train loss: 0.255388
Epoch :3 | step 3000, training accuracy: 0.930000, train loss: 0.424817
Epoch :3 | step 3100, training accuracy: 0.980000, train loss: 0.168380
Epoch :3 | step 3200, training accuracy: 0.940000, train loss: 0.292462
Epoch :3 | step 3300, training accuracy: 0.960000, train loss: 0.195383
Epoch :3 | step 3400, training accuracy: 0.970000, train loss: 0.433935
Epoch :3 | step 3500, training accuracy: 0.970000, train loss: 0.341178
Epoch :3 | step 3600, training accuracy: 0.960000, train loss: 0.204085
Epoch :3 | step 3700, training accuracy: 0.950000, train loss: 0.465894
Epoch :3 | step 3800, training accuracy: 0.970000, train loss: 0

Epoch :5 | step 400, training accuracy: 0.970000, train loss: 0.079554
Epoch :5 | step 500, training accuracy: 0.970000, train loss: 0.188524
Epoch :5 | step 600, training accuracy: 0.970000, train loss: 0.155434
Epoch :5 | step 700, training accuracy: 0.990000, train loss: 0.110556
Epoch :5 | step 800, training accuracy: 0.950000, train loss: 0.284303
Epoch :5 | step 900, training accuracy: 0.970000, train loss: 0.240928
Epoch :5 | step 1000, training accuracy: 0.960000, train loss: 0.231810
Epoch :5 | step 1100, training accuracy: 0.980000, train loss: 0.158629
Epoch :5 | step 1200, training accuracy: 0.960000, train loss: 0.519485
Epoch :5 | step 1300, training accuracy: 0.960000, train loss: 0.183919
Epoch :5 | step 1400, training accuracy: 0.970000, train loss: 0.211090
Epoch :5 | step 1500, training accuracy: 0.960000, train loss: 0.110421
Epoch :5 | step 1600, training accuracy: 0.960000, train loss: 0.668822
Epoch :5 | step 1700, training accuracy: 0.940000, train loss: 0.41163

Epoch :6 | step 5100, training accuracy: 0.980000, train loss: 0.217607
Epoch :6 | step 5200, training accuracy: 0.980000, train loss: 0.082649
Epoch :6 | step 5300, training accuracy: 0.970000, train loss: 0.159168
Epoch :6 | step 5400, training accuracy: 0.960000, train loss: 0.181510
Epoch :6 | step 5500, training accuracy: 0.970000, train loss: 0.151417
Epoch :6 | step 5600, training accuracy: 0.980000, train loss: 0.176315
Epoch :6 | step 5700, training accuracy: 0.980000, train loss: 0.306074
Epoch :6 | step 5800, training accuracy: 0.960000, train loss: 0.122553
Epoch :6 | step 5900, training accuracy: 1.000000, train loss: 0.039121
Epoch :6 | step 6000, training accuracy: 0.970000, train loss: 0.310007
Epoch :6 | step 6100, training accuracy: 0.960000, train loss: 0.171784
Epoch :6 | step 6200, training accuracy: 0.970000, train loss: 0.241155
Epoch :6 | step 6300, training accuracy: 0.990000, train loss: 0.215127
Epoch :6 | step 6400, training accuracy: 0.960000, train loss: 0

...

Epoch :41 | step 4800, training accuracy: 1.000000, train loss: 0.004552
Epoch :41 | step 4900, training accuracy: 0.990000, train loss: 0.057505
Epoch :41 | step 5000, training accuracy: 1.000000, train loss: 0.001833
Epoch :41 | step 5100, training accuracy: 1.000000, train loss: 0.004136
Epoch :41 | step 5200, training accuracy: 1.000000, train loss: 0.003660
Epoch :41 | step 5300, training accuracy: 1.000000, train loss: 0.005274
Epoch :41 | step 5400, training accuracy: 1.000000, train loss: 0.006316
Epoch :41 | step 5500, training accuracy: 1.000000, train loss: 0.001652
Epoch :41 | step 5600, training accuracy: 1.000000, train loss: 0.002069
Epoch :41 | step 5700, training accuracy: 1.000000, train loss: 0.008322
Epoch :41 | step 5800, training accuracy: 1.000000, train loss: 0.002385
Epoch :41 | step 5900, training accuracy: 0.990000, train loss: 0.035987
Epoch :41 | step 6000, training accuracy: 0.990000, train loss: 0.031293
Epoch :41 | step 6100, training accuracy: 1.000000,

Epoch :43 | step 2500, training accuracy: 1.000000, train loss: 0.002979
Epoch :43 | step 2600, training accuracy: 0.980000, train loss: 0.068890
Epoch :43 | step 2700, training accuracy: 1.000000, train loss: 0.008564
Epoch :43 | step 2800, training accuracy: 0.990000, train loss: 0.034141
Epoch :43 | step 2900, training accuracy: 1.000000, train loss: 0.000756
Epoch :43 | step 3000, training accuracy: 1.000000, train loss: 0.002218
Epoch :43 | step 3100, training accuracy: 1.000000, train loss: 0.000827
Epoch :43 | step 3200, training accuracy: 1.000000, train loss: 0.004109
Epoch :43 | step 3300, training accuracy: 1.000000, train loss: 0.001286
Epoch :43 | step 3400, training accuracy: 1.000000, train loss: 0.004671
Epoch :43 | step 3500, training accuracy: 1.000000, train loss: 0.004006
Epoch :43 | step 3600, training accuracy: 1.000000, train loss: 0.003172
Epoch :43 | step 3700, training accuracy: 1.000000, train loss: 0.002892
Epoch :43 | step 3800, training accuracy: 1.000000,

Epoch :45 | step 200, training accuracy: 1.000000, train loss: 0.005812
Epoch :45 | step 300, training accuracy: 1.000000, train loss: 0.009861
Epoch :45 | step 400, training accuracy: 1.000000, train loss: 0.000741
Epoch :45 | step 500, training accuracy: 1.000000, train loss: 0.005932
Epoch :45 | step 600, training accuracy: 0.990000, train loss: 0.010781
Epoch :45 | step 700, training accuracy: 1.000000, train loss: 0.000511
Epoch :45 | step 800, training accuracy: 0.990000, train loss: 0.024201
Epoch :45 | step 900, training accuracy: 1.000000, train loss: 0.003928
Epoch :45 | step 1000, training accuracy: 0.990000, train loss: 0.072439
Epoch :45 | step 1100, training accuracy: 1.000000, train loss: 0.002722
Epoch :45 | step 1200, training accuracy: 1.000000, train loss: 0.003287
Epoch :45 | step 1300, training accuracy: 1.000000, train loss: 0.001733
Epoch :45 | step 1400, training accuracy: 1.000000, train loss: 0.000346
Epoch :45 | step 1500, training accuracy: 0.990000, train l

Epoch :46 | step 4700, training accuracy: 1.000000, train loss: 0.002004
Epoch :46 | step 4800, training accuracy: 1.000000, train loss: 0.001310
Epoch :46 | step 4900, training accuracy: 1.000000, train loss: 0.006538
Epoch :46 | step 5000, training accuracy: 0.990000, train loss: 0.019682
Epoch :46 | step 5100, training accuracy: 1.000000, train loss: 0.005828
Epoch :46 | step 5200, training accuracy: 1.000000, train loss: 0.002185
Epoch :46 | step 5300, training accuracy: 0.990000, train loss: 0.014975
Epoch :46 | step 5400, training accuracy: 1.000000, train loss: 0.001792
Epoch :46 | step 5500, training accuracy: 1.000000, train loss: 0.003315
Epoch :46 | step 5600, training accuracy: 1.000000, train loss: 0.000872
Epoch :46 | step 5700, training accuracy: 1.000000, train loss: 0.000405
Epoch :46 | step 5800, training accuracy: 1.000000, train loss: 0.003054
Epoch :46 | step 5900, training accuracy: 1.000000, train loss: 0.002088
Epoch :46 | step 6000, training accuracy: 1.000000,

Epoch :48 | step 2400, training accuracy: 0.990000, train loss: 0.010273
Epoch :48 | step 2500, training accuracy: 1.000000, train loss: 0.002429
Epoch :48 | step 2600, training accuracy: 1.000000, train loss: 0.001755
Epoch :48 | step 2700, training accuracy: 1.000000, train loss: 0.005939
Epoch :48 | step 2800, training accuracy: 1.000000, train loss: 0.004435
Epoch :48 | step 2900, training accuracy: 0.990000, train loss: 0.088242
Epoch :48 | step 3000, training accuracy: 0.990000, train loss: 0.025098
Epoch :48 | step 3100, training accuracy: 1.000000, train loss: 0.005173
Epoch :48 | step 3200, training accuracy: 1.000000, train loss: 0.001051
Epoch :48 | step 3300, training accuracy: 1.000000, train loss: 0.006579
Epoch :48 | step 3400, training accuracy: 0.980000, train loss: 0.051153
Epoch :48 | step 3500, training accuracy: 1.000000, train loss: 0.001832
Epoch :48 | step 3600, training accuracy: 1.000000, train loss: 0.000493
Epoch :48 | step 3700, training accuracy: 1.000000,

In [18]:
W2_cho2 = tf.Variable(tf.random_normal([512,19],stddev=0.01))
W2_jung2 = tf.Variable(tf.random_normal([512,21],stddev=0.01))
W2_jong2 = tf.Variable(tf.random_normal([512,28],stddev=0.01))

print(fc)
fc2_cho2 = tf.matmul(fc, W2_cho2)
cho_logit = fc2_cho2
bn2_cho2 = tf.layers.batch_normalization(fc2_cho2, training=is_training)
fc2_cho2 = tf.nn.relu(bn2_cho2)
fc2_cho2 = tf.nn.dropout(fc2_cho2, keep_prob)

fc2_jung2 = tf.matmul(fc, W2_jung2)
jung_logit = fc2_jung2
bn2_jung2 = tf.layers.batch_normalization(fc2_jung2, training=is_training)
fc2_jung2 = tf.nn.relu(bn2_jung2)
fc2_jung2 = tf.nn.dropout(fc2_jung2, keep_prob)

fc2_jong2 = tf.matmul(fc, W2_jong2)
jong_logit = fc2_jong2
bn2_jong2 = tf.layers.batch_normalization(fc2_jong2, training=is_training)
fc2_jong2 = tf.nn.relu(bn2_jong2)
fc2_jong2 = tf.nn.dropout(fc2_jong2, keep_prob)

Tensor("Reshape:0", shape=(?, 512), dtype=float32)


In [19]:
W_cho3 = tf.Variable(tf.random_normal([19,2350],stddev=0.01))
W_jung3 = tf.Variable(tf.random_normal([21,2350],stddev=0.01))
W_jong3 = tf.Variable(tf.random_normal([28,2350],stddev=0.01))

fc3_cho2 = tf.matmul(fc2_cho2, W_cho3)
fc3_jung2 = tf.matmul(fc2_jung2, W_jung3)
fc3_jong2 = tf.matmul(fc2_jong2, W_jong3)

logit = fc3_cho2 + fc3_jung2 + fc3_jong2

In [20]:
cross_entropy2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels)
loss = tf.reduce_mean(cross_entropy2)

cho_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=cho_logit, labels=input_label_cho)
jung_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=jung_logit, labels=input_label_jung)
jong_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=jong_logit, labels=input_label_jong)

cho_loss = tf.reduce_mean(cho_cross_entropy)
jung_loss = tf.reduce_mean(jung_cross_entropy)
jong_loss = tf.reduce_mean(jong_cross_entropy)

loss = loss + 3*((0.4*cho_loss) + 0.4*(jung_loss) + 0.2*(jong_loss))

update_ops2 = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops2):
    optimizer2 = tf.train.AdamOptimizer(0.001).minimize(loss)
loss_summ2=tf.summary.scalar("Loss",loss)
        
correct_prediction2 = tf.equal(tf.argmax(logit,1), input_labels)
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
test_accuracy2 = tf.reduce_sum(tf.cast(correct_prediction2, tf.float32))
acc_summ2=tf.summary.scalar("Accuracy",accuracy2)

In [21]:
def decode2(serialized_example):
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    
    return image,label

In [22]:
batch_size=100
#test_file_name = "Shuffled_PE92_Test.tfrecord"
#train_file_name = "Shuffled_PE92_Train.tfrecord"

test_file_name = "Shuffled_with_alphabet_PE92_Test.tfrecord"
train_file_name = "Shuffled_with_alphabet_PE92_Train.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [23]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary3 = tf.summary.merge([loss_summ2, acc_summ2])
    train_writer = tf.summary.FileWriter("./logs/CNNV3/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV3/PE92/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_label, train_cho, train_jung, train_jong])
            
            sess.run(optimizer2,feed_dict={input_images: batch_images, input_labels: batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary3, feed_dict={input_images: batch_images, input_labels:batch_labels, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/PE92/CNNV3/CNNV3_PE92.ckpt")
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_label, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss=sess.run([test_accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong,keep_prob:1.0, is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_labels)
        loss_sum=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
       
    print("Training is done.")

    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 16.784590
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 25.133587
Epoch :0 | step 200, training accuracy: 0.040000, train loss: 10.584522
Epoch :0 | step 300, training accuracy: 0.050000, train loss: 8.914495
Epoch :0 | step 400, training accuracy: 0.110000, train loss: 6.907438
Epoch :0 | step 500, training accuracy: 0.210000, train loss: 5.137005
Epoch :0 | step 600, training accuracy: 0.120000, train loss: 6.197278
Epoch :0 | step 700, training accuracy: 0.400000, train loss: 3.974199
Epoch :0 | step 800, training accuracy: 0.180000, train loss: 5.064667
Epoch :0 | step 900, training accuracy: 0.180000, train loss: 6.245075
Epoch :0 | step 1000, training accuracy: 0.580000, train loss: 2.727718
Epoch :0 | step 1100, training accuracy: 0.800000, train loss: 2.041603
Epoch :0 | step 1200, training accuracy: 0.640000, train loss: 2.583053
Epoch :0 | step 1300, training accuracy: 0.570000

Epoch :5 | step 500, training accuracy: 0.970000, train loss: 0.157262
Epoch :5 | step 600, training accuracy: 0.960000, train loss: 0.173963
Epoch :5 | step 700, training accuracy: 0.990000, train loss: 0.089216
Epoch :5 | step 800, training accuracy: 0.990000, train loss: 0.108146
Epoch :5 | step 900, training accuracy: 1.000000, train loss: 0.083407
Epoch :5 | step 1000, training accuracy: 0.990000, train loss: 0.065285
Epoch :5 | step 1100, training accuracy: 0.990000, train loss: 0.126387
Epoch :5 | step 1200, training accuracy: 1.000000, train loss: 0.114512
Epoch :5 | step 1300, training accuracy: 0.970000, train loss: 0.359896
Epoch :5 | step 1400, training accuracy: 0.970000, train loss: 0.166620
Epoch :5 | step 1500, training accuracy: 1.000000, train loss: 0.039209
Epoch :5 | step 1600, training accuracy: 0.980000, train loss: 0.131382
Epoch :5 | step 1700, training accuracy: 0.960000, train loss: 0.286941
Epoch :5 | step 1800, training accuracy: 0.990000, train loss: 0.0749

Epoch :10 | step 1000, training accuracy: 0.990000, train loss: 0.195790
Epoch :10 | step 1100, training accuracy: 0.980000, train loss: 0.070529
Epoch :10 | step 1200, training accuracy: 1.000000, train loss: 0.018975
Epoch :10 | step 1300, training accuracy: 0.990000, train loss: 0.050208
Epoch :10 | step 1400, training accuracy: 0.990000, train loss: 0.055746
Epoch :10 | step 1500, training accuracy: 1.000000, train loss: 0.057445
Epoch :10 | step 1600, training accuracy: 1.000000, train loss: 0.035342
Epoch :10 | step 1700, training accuracy: 1.000000, train loss: 0.006200
Epoch :10 | step 1800, training accuracy: 1.000000, train loss: 0.011545
Epoch :10 | step 1900, training accuracy: 1.000000, train loss: 0.018906
Epoch :10 | step 2000, training accuracy: 1.000000, train loss: 0.008741
Time : 0 : 40: :14.582106
Test Acc : 0.975172, Test Loss :0.000284
Epoch :11 | step 0, training accuracy: 0.990000, train loss: 0.042718
Epoch :11 | step 100, training accuracy: 0.970000, train los

Epoch :15 | step 1400, training accuracy: 0.990000, train loss: 0.024558
Epoch :15 | step 1500, training accuracy: 1.000000, train loss: 0.005216
Epoch :15 | step 1600, training accuracy: 1.000000, train loss: 0.007757
Epoch :15 | step 1700, training accuracy: 1.000000, train loss: 0.041648
Epoch :15 | step 1800, training accuracy: 1.000000, train loss: 0.002214
Epoch :15 | step 1900, training accuracy: 1.000000, train loss: 0.006905
Epoch :15 | step 2000, training accuracy: 1.000000, train loss: 0.024726
Time : 0 : 58: :36.665142
Test Acc : 0.974018, Test Loss :0.015746
Epoch :16 | step 0, training accuracy: 1.000000, train loss: 0.007409
Epoch :16 | step 100, training accuracy: 1.000000, train loss: 0.015463
Epoch :16 | step 200, training accuracy: 1.000000, train loss: 0.008944
Epoch :16 | step 300, training accuracy: 1.000000, train loss: 0.007189
Epoch :16 | step 400, training accuracy: 0.990000, train loss: 0.022201
Epoch :16 | step 500, training accuracy: 1.000000, train loss: 0

Epoch :20 | step 1800, training accuracy: 1.000000, train loss: 0.015911
Epoch :20 | step 1900, training accuracy: 1.000000, train loss: 0.012120
Epoch :20 | step 2000, training accuracy: 1.000000, train loss: 0.009636
Time : 1 : 16: :58.575167
Test Acc : 0.975599, Test Loss :0.000427
Epoch :21 | step 0, training accuracy: 1.000000, train loss: 0.085788
Epoch :21 | step 100, training accuracy: 1.000000, train loss: 0.001257
Epoch :21 | step 200, training accuracy: 1.000000, train loss: 0.008547
Epoch :21 | step 300, training accuracy: 1.000000, train loss: 0.003694
Epoch :21 | step 400, training accuracy: 1.000000, train loss: 0.004182
Epoch :21 | step 500, training accuracy: 1.000000, train loss: 0.005352
Epoch :21 | step 600, training accuracy: 0.990000, train loss: 0.045543
Epoch :21 | step 700, training accuracy: 1.000000, train loss: 0.004861
Epoch :21 | step 800, training accuracy: 1.000000, train loss: 0.017084
Epoch :21 | step 900, training accuracy: 0.990000, train loss: 0.029

Epoch :26 | step 100, training accuracy: 1.000000, train loss: 0.006120
Epoch :26 | step 200, training accuracy: 1.000000, train loss: 0.001277
Epoch :26 | step 300, training accuracy: 0.990000, train loss: 0.013499
Epoch :26 | step 400, training accuracy: 1.000000, train loss: 0.001798
Epoch :26 | step 500, training accuracy: 1.000000, train loss: 0.001836
Epoch :26 | step 600, training accuracy: 1.000000, train loss: 0.004837
Epoch :26 | step 700, training accuracy: 1.000000, train loss: 0.005382
Epoch :26 | step 800, training accuracy: 1.000000, train loss: 0.002101
Epoch :26 | step 900, training accuracy: 1.000000, train loss: 0.001771
Epoch :26 | step 1000, training accuracy: 0.990000, train loss: 0.014000
Epoch :26 | step 1100, training accuracy: 1.000000, train loss: 0.003211
Epoch :26 | step 1200, training accuracy: 1.000000, train loss: 0.005993
Epoch :26 | step 1300, training accuracy: 0.990000, train loss: 0.020236
Epoch :26 | step 1400, training accuracy: 1.000000, train lo

Epoch :31 | step 500, training accuracy: 1.000000, train loss: 0.001422
Epoch :31 | step 600, training accuracy: 1.000000, train loss: 0.006674
Epoch :31 | step 700, training accuracy: 1.000000, train loss: 0.000967
Epoch :31 | step 800, training accuracy: 1.000000, train loss: 0.001617
Epoch :31 | step 900, training accuracy: 1.000000, train loss: 0.001674
Epoch :31 | step 1000, training accuracy: 1.000000, train loss: 0.001166
Epoch :31 | step 1100, training accuracy: 1.000000, train loss: 0.000451
Epoch :31 | step 1200, training accuracy: 1.000000, train loss: 0.025945
Epoch :31 | step 1300, training accuracy: 1.000000, train loss: 0.005847
Epoch :31 | step 1400, training accuracy: 1.000000, train loss: 0.025671
Epoch :31 | step 1500, training accuracy: 0.990000, train loss: 0.112455
Epoch :31 | step 1600, training accuracy: 1.000000, train loss: 0.001704
Epoch :31 | step 1700, training accuracy: 1.000000, train loss: 0.013263
Epoch :31 | step 1800, training accuracy: 1.000000, trai

Epoch :36 | step 900, training accuracy: 1.000000, train loss: 0.000801
Epoch :36 | step 1000, training accuracy: 1.000000, train loss: 0.003152
Epoch :36 | step 1100, training accuracy: 1.000000, train loss: 0.004342
Epoch :36 | step 1200, training accuracy: 1.000000, train loss: 0.005315
Epoch :36 | step 1300, training accuracy: 1.000000, train loss: 0.002176
Epoch :36 | step 1400, training accuracy: 1.000000, train loss: 0.010668
Epoch :36 | step 1500, training accuracy: 1.000000, train loss: 0.001042
Epoch :36 | step 1600, training accuracy: 1.000000, train loss: 0.013966
Epoch :36 | step 1700, training accuracy: 1.000000, train loss: 0.000519
Epoch :36 | step 1800, training accuracy: 1.000000, train loss: 0.002721
Epoch :36 | step 1900, training accuracy: 1.000000, train loss: 0.000408
Epoch :36 | step 2000, training accuracy: 1.000000, train loss: 0.002959
Time : 2 : 15: :45.055870
Test Acc : 0.976069, Test Loss :0.000943
Epoch :37 | step 0, training accuracy: 1.000000, train los

Epoch :41 | step 1300, training accuracy: 1.000000, train loss: 0.001055
Epoch :41 | step 1400, training accuracy: 1.000000, train loss: 0.001257
Epoch :41 | step 1500, training accuracy: 1.000000, train loss: 0.003980
Epoch :41 | step 1600, training accuracy: 1.000000, train loss: 0.002781
Epoch :41 | step 1700, training accuracy: 1.000000, train loss: 0.001758
Epoch :41 | step 1800, training accuracy: 1.000000, train loss: 0.004600
Epoch :41 | step 1900, training accuracy: 1.000000, train loss: 0.006957
Epoch :41 | step 2000, training accuracy: 1.000000, train loss: 0.000870
Time : 2 : 34: :6.983897
Test Acc : 0.976710, Test Loss :0.368243
Epoch :42 | step 0, training accuracy: 1.000000, train loss: 0.000423
Epoch :42 | step 100, training accuracy: 1.000000, train loss: 0.000389
Epoch :42 | step 200, training accuracy: 1.000000, train loss: 0.002685
Epoch :42 | step 300, training accuracy: 1.000000, train loss: 0.000672
Epoch :42 | step 400, training accuracy: 1.000000, train loss: 0

Epoch :46 | step 1700, training accuracy: 1.000000, train loss: 0.000188
Epoch :46 | step 1800, training accuracy: 1.000000, train loss: 0.004209
Epoch :46 | step 1900, training accuracy: 1.000000, train loss: 0.013567
Epoch :46 | step 2000, training accuracy: 1.000000, train loss: 0.003501
Time : 2 : 52: :29.075933
Test Acc : 0.977565, Test Loss :0.000867
Epoch :47 | step 0, training accuracy: 1.000000, train loss: 0.000174
Epoch :47 | step 100, training accuracy: 1.000000, train loss: 0.001250
Epoch :47 | step 200, training accuracy: 1.000000, train loss: 0.008186
Epoch :47 | step 300, training accuracy: 1.000000, train loss: 0.000488
Epoch :47 | step 400, training accuracy: 1.000000, train loss: 0.000470
Epoch :47 | step 500, training accuracy: 1.000000, train loss: 0.001292
Epoch :47 | step 600, training accuracy: 1.000000, train loss: 0.001080
Epoch :47 | step 700, training accuracy: 1.000000, train loss: 0.011987
Epoch :47 | step 800, training accuracy: 1.000000, train loss: 0.00

In [24]:
train_file_name = "Shuffled_HanDB_Train.tfrecord"
test_file_name = "Shuffled_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [25]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary4 = tf.summary.merge([loss_summ2, acc_summ2])
    train_writer = tf.summary.FileWriter("./logs/CNNV3/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV3/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_label, train_cho, train_jung, train_jong])
            
            sess.run(optimizer2,feed_dict={input_images: batch_images, input_labels: batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary4, feed_dict={input_images: batch_images, input_labels:batch_labels, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/HanDB/CNNV3/CNNV3_PE92.ckpt")
        
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_label, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss=sess.run([test_accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong,keep_prob:1.0, is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_labels)
        loss_sum=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
       
    print("Training is done.")

    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :6658


InvalidArgumentError: Name: <unknown>, Feature: cho_label (data type: int64) is required but could not be found.
	 [[Node: ParseSingleExample/ParseExample/ParseExample = ParseExample[Ndense=7, Nsparse=0, Tdense=[DT_INT64, DT_INT64, DT_STRING, DT_INT64, DT_INT64, DT_INT64, DT_INT64], dense_shapes=[[], [], [], [], [], [], []], sparse_types=[]](ParseSingleExample/ExpandDims, ParseSingleExample/ParseExample/ParseExample/names, ParseSingleExample/ParseExample/ParseExample/dense_keys_0, ParseSingleExample/ParseExample/ParseExample/dense_keys_1, ParseSingleExample/ParseExample/ParseExample/dense_keys_2, ParseSingleExample/ParseExample/ParseExample/dense_keys_3, ParseSingleExample/ParseExample/ParseExample/dense_keys_4, ParseSingleExample/ParseExample/ParseExample/dense_keys_5, ParseSingleExample/ParseExample/ParseExample/dense_keys_6, ParseSingleExample/ParseExample/Const, ParseSingleExample/ParseExample/Const_1, ParseSingleExample/ParseExample/Const_2, ParseSingleExample/ParseExample/Const_3